<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<h1>Algorísmica Avançada</h1>
<h2>Pràctica 1 - Exploració: Grafs, DFS, BFS, Dijkstra</h2>
</center>
</div>

In [61]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<div class="alert alert-info">
<h2>Avís general per a les Pràctiques de Laboratori</h2>

És molt important <b>comentar tot el vostre codi</b>. Comentar el codi no només facilita a altres persones que el llegeixin la compressió del que esteu fent, sinó també a vosaltres mateixos durant el procés de disseny i implementació. D'aquesta forma podreu entendre el vostre propi codi encara que faci molt temps que no el consulteu. A més, a l'hora de ser corregits, un codi comentat s'entén molt millor, la qual cosa ajuda a traslladar al professor les idees que teníeu a l'intentar implementar-lo i encara més en el cas que no ho hagueu pogut acabar-ho amb èxit. No cal comentar cada línia, però si el que fa cada part.

En segon lloc, és molt important <b>respectar les capçaleres de les funcions i estructura de la funció ```return```</b> que s'indiquin a l'enunciat. Tot i que llegim i corregim els vostres codis en detall, fem servir correctors automàtics per guiar i agilitzar el procés i saber quines solucións funcionen i quines no ja d'entrada. 

Axí doncs, si es demana que la funció tingui la següent estructura: 

```python
def exercici1(param1, param2):
    # implementació de la solució
    return resultat 
```

És important que en el notebook existeixi aquesta funció i que no canvieu el nom dels paràmetres ni el tipus de dada retornada.
    
</div>




In [62]:
from matplotlib import pyplot as plt
import networkx as nx
from maze import *
import heapq as hq
import numpy as np

<div class="alert alert-success">
<h1>Problema 3: Checkpoint (2,5p)</h1>
<p>
    Implementeu una funció anomenada <b>checkpoint()</b> que, donat un graf, un node origen, un node destí i un node extra, trobi el camí més curt des del node origen fins el node destí passant pel node extra. Com en l'exercici anterior, tindrem un diccionari de punts on aplicarem una penalització.<br>
</p>    
    
</div>

In [63]:
def bfs(G, origin):
    """
    Params
    ======    
    :G: Graf. Ha de ser un objecte de la classe nx.Graph
    :origin: Índex del node orígen
    
    Returns
    =======
    :visited: El conjunt de nodes visitats durant l'exploració BFS en l'ordre en que han estat explorats
    """
    visited = []

    # Comenzamos en el nodo origen y lo añadimos a la cola.
    queue = [origin]

    while queue:

        # Tomamos el primer elemento de la cola.
        current = queue.pop(0)

        # Si no lo hemos visitado, lo añadimos a la lista de visitados.
        # Luego, añadimos todos sus vecinos a la cola y continuamos.
        if current not in visited:

            visited.append(current)

            for nei in G.neighbors(current):
                queue.append(nei)
    
    return visited

'''
    Función auxiliar que funciona exactamente igual a holes, pero devuelve cost_to en vez de el coste y camino.
'''

def holes_aux(G, origin, holes_dct={}):
    """
    Params
    ======
    :G: Graf del qual en volem extreure el camí mínim. Ha de ser un objecte de la classe nx.Graph
    :origin: Índex del node orígen
    :holes_dct: Un diccionari del tipus {node: penalització}. Passar per cada node del diccionari té un cost diferent a 1.
    
    Returns
    =======
    :cost_to: Lista de distancias desde el nodo origen hasta el resto.
    """

    cost_to = {}
    visited = set()
    
    for node in G.nodes():
        cost_to[node] = (None, np.inf) # Seguiré la estructura de cost_to[current] = (previous, cost)
        
    cost_to[origin] = (None, 0) #Puesto que comenzamos en el nodo de origen, no tiene previo y su coste es cero.

    # Creemos ahora el heap.
    heap = []
    hq.heapify(heap)
    hq.heappush(heap, (0, origin)) # Para que el heap pueda ordenar en base al coste, seguiré la estructura (coste, current).

    # Y utilicemos el heap para explorar todo el grafo con costes mínimos.
    while heap:
        current_cost, current_node = hq.heappop(heap) # Obtengo el siguiente camino de coste menor.

        if current_node in visited:
            continue

        visited.add(current_node)

        for nei in G.neighbors(current_node):

            if nei in visited:
                continue

            
            weight = 1 if nei not in holes_dct else holes_dct[nei] # Si es un nodo con penalización, añadimos su penalización.
            alternative_path = current_cost + weight

            if alternative_path < cost_to[nei][1]:
                cost_to[nei] = (current_node, alternative_path)
                hq.heappush(heap, (alternative_path, nei))

    # Al vaciar el heap, tendremos el diccionario cost_to con todos los costes a todos los nodos. Solo queda reconstruir el camino.

    return cost_to

def checkpoint(G, origin, destination, extra, holes_dct={}):
    """
    Params
    ======
    :G: Graf del qual en volem extreure el camí mínim. Ha de ser un objecte de la classe nx.Graph
    :origin: Índex del node orígen
    :destination: Índex del node destí
    :extra: Índex d'un node extra per on ha de passar el camí
    :holes_dct: Un diccionari del tipus {node: penalització}. Passar per cada node del diccionari té un cost diferent a 1.
    
    Returns
    =======
    :path: Una llista de nodes del camí més curt entre els nodes 'origin' i 'destination' que passa per 'extra'.
    :cost: Un enter amb el cost de recórrer el camí, incloent-hi les penalitzacions.
    """
    path = []
    cost = 0

    # Comprobemos que sea posible hacer un camino desde origin hasta destination.
    if destination not in bfs(G, origin):
        return [], -1
    
    # Obtenemos el diccionario con todos los costes desde extra. Este diccionario tiene a su vez la distancia mínima hasta origen
    # y la distancia mínima hasta destination, así como la información para construir el camino.
    # Esto es mejor que llamar dos holes.
    cost_to = holes_aux(G, extra, holes_dct)

    # Creemos el camino desde origen hasta extra.
    path_from_origin_to_extra = []
    cost_from_origin_to_extra = cost_to[origin][1]
    
    # Empezamos el origin, calculamos su previo, vemos el previo del previo...
    # Así hasta llegar a extra. Donde habremos creado el camino hasta extra en orden.
    pointer_to_node = origin
    while pointer_to_node != extra:
        path_from_origin_to_extra.append(pointer_to_node)
        pointer_to_node = cost_to[pointer_to_node][0]


    # camino desde extra hasta destination.
    path_from_extra_to_destination = []
    cost_from_extra_to_destination = cost_to[destination][1]
    pointer_to_node = destination
    while pointer_to_node != extra:
        pointer_to_node = cost_to[pointer_to_node][0]
        path_from_extra_to_destination.append(pointer_to_node)
    
    #La diferencia es que en este caso, los elementos se añaden en orden inverso y por tanto tenemos que arreglarlo.
    path_from_extra_to_destination.reverse()
    
    path = path_from_origin_to_extra + path_from_extra_to_destination # Sumamos los dos caminos.
    cost = cost_from_origin_to_extra + cost_from_extra_to_destination # Sumamos los costos.
    
    path.append(destination)

    return path, cost

<div class="alert alert-info">
<center>
  <h1>Entrega</h1>
</center>

L'entrega de la prova de pràctica es farà amb el mòdul de <b>ub-grader</b>. Per instal·lar-lo feu servir la següent comanda a terminal:

```
python pip install ub-grader --upgrade
```

O una cel·la del notebook amb:

```
!pip install ub-grader --upgrade
```
<br>

Després, a la cel·la següent canvieu els niubs i noms pels vostres. Executeu-la per generar el vostre report del problema. En aquest cas mostrem quins són els inputs de la funció als tests, però això no passarà al dia del examen. Finalment, el report es generarà al directori d'aquest notebook amb nom ```report_Membre1_Membre2_ex3.enc```, pugeu aquest fitxer al campus virtual una vegada acabada la prova.

És imprescindible que respecteu el format de les funcions i que el vostre codi estigui degudament comentat.<br><br>

</div>

In [64]:
!pip install ub-grader --upgrade

In [65]:
from ub_grader import init_students, load_spec, grade

# Funció wrapper pel exercici
def wrapper_ex3(n,m,seed,A,B,E,H):
    H = {int(k) : v for k, v in H.items()}
    maze = Maze(n, m, seed=seed)
    G = maze.maze_graph()
    path, cost = checkpoint(G, A, B, E, H)
    cost += 1
    return {
        "cost" : cost,
        "path" : path
    }

init_students([
    {"niub": "20921854", "nombre": "KARIM", "apellidos": "BOUJANA MARCUCCI"},
    {"niub": "22303610", "nombre": "BESAY", "apellidos": "DÍAZ RIBAS"}
])


load_spec(r"file:///./specs/ex3.json")

result = grade(
    wrapper_ex3,
    signing_key_path=None,  # Path to Ed25519 private key if signing
)

[TEST t1] Input [6, 6, 17, 30, 35, 8, {}]
[TEST t2] Input [6, 6, 17, 30, 35, 8, {'13': 3, '28': 5}]
[TEST t3] Input [6, 6, 17, 30, 35, 8, {'13': 10, '28': 20}]
Grade: 10.0 / 10.0
